In [ ]:
!pip install google-play-scraper
!pip install nlu
!pip install pyspark

     |████████████████████████████████| 52 kB 1.0 MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.3-py3-none-any.whl size=24394 sha256=fd351b0ebec74ec9e9f31bb5382f062ea0707dad2603e1f7ec883ced37230478
  Stored in directory: /root/.cache/pip/wheels/81/37/0b/4a14be55b449a048cd93d79930b1a980dee7896480defa1923
Successfully built google-play-scraper


In [ ]:
import itertools
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud
from google_play_scraper import Sort, reviews_all
import nlu

In [ ]:
result = reviews_all(
    'xyz.sheba.managerapp',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
    filter_score_with= None # defaults to None(means all score)
)

In [ ]:
playstore_comments = pd.DataFrame(np.array(result), columns = ['content'])

playstore_comments = playstore_comments.join(pd.DataFrame(playstore_comments.pop('content').tolist()))



len(playstore_comments)

3695

In [ ]:
len(playstore_comments)

3695

In [ ]:
playstore_commentsdf = playstore_comments['content']

In [ ]:
print(playstore_commentsdf)

0       Assalamu alaikum this software is really very ...
1       App is good but transactions has a bug. If we ...
2       It's an amazing app... I use this app 5 months...
3       Bad app, bad service. So far none of the compl...
4       It was doing well but at present its services ...
                              ...                        
3684                                                 Good
3685                                                 Nice
3686                                                 Good
3687                                                 nice
3688                                           Asmotullah
Name: content, Length: 3689, dtype: object


In [ ]:
playstore_commentsdf.to_csv("Playstore_main_file.csv")

In [ ]:
Gplay_Reviews = playstore_comments['content'].dropna()

In [ ]:
print(Gplay_Reviews)

0       Assalamu alaikum this software is really very ...
1       App is good but transactions has a bug. If we ...
2       It's an amazing app... I use this app 5 months...
3       Bad app, bad service. So far none of the compl...
4       It was doing well but at present its services ...
                              ...                        
3684                                                 Good
3685                                                 Nice
3686                                                 Good
3687                                                 nice
3688                                           Asmotullah
Name: content, Length: 3689, dtype: object


In [ ]:
Gplay_Reviews = Gplay_Reviews.str.lower()
Gplay_Reviews.head()

0    assalamu alaikum this software is really very ...
1    app is good but transactions has a bug. if we ...
2    it's an amazing app... i use this app 5 months...
3    bad app, bad service. so far none of the compl...
4    it was doing well but at present its services ...
Name: content, dtype: object

In [ ]:
Gplay_Reviews = Gplay_Reviews.str.replace('[^\w\s]', '')
Gplay_Reviews = Gplay_Reviews.str.replace('\d', '')
Gplay_Reviews

0       assalamu alaikum this software is really very ...
1       app is good but transactions has a bug if we e...
2       its an amazing app i use this app  monthsvery ...
3       bad app bad service so far none of the complai...
4       it was doing well but at present its services ...
                              ...                        
3684                                                 good
3685                                                 nice
3686                                                 good
3687                                                 nice
3688                                           asmotullah
Name: content, Length: 3689, dtype: object

In [ ]:
stopword_df = nlu.load('bn.stopwords').predict(Gplay_Reviews)

stopwords_bn download started this may take some time.
Approximate size to download 1.9 KB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [ ]:
Gplay_stopword_df = nlu.load('bn.stopwords').predict(Gplay_Reviews)

stopwords_bn download started this may take some time.
Approximate size to download 1.9 KB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [ ]:
print(Gplay_stopword_df)

     stopword_less       token
0         assalamu    assalamu
0          alaikum     alaikum
0             this        this
0         software    software
0               is          is
...            ...         ...
3684          good        good
3685          nice        nice
3686          good        good
3687          nice        nice
3688    asmotullah  asmotullah

[21350 rows x 2 columns]


In [ ]:
Gplay_Reviews_only_eng = Gplay_Reviews.apply(lambda x: " ".join(x for x in str(x).split() if x not in Gplay_stopword_df))

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
sw = stopwords.words('english')

In [ ]:
append_list = ['ekhono','plz','din','ase','te','ki','te','oen','jay','yes','amr','amar','nai','kintu','sudu','kore','kono','ekbare','jabe','babe','ar','s','seta','fon','amn','amon','emon','emn','hobe','hbe','chaie']

In [ ]:
for text in append_list:
  sw.append(text)

In [ ]:
Gplay_Reviews_all = Gplay_Reviews_only_eng.apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
Gplay_Reviews_token = Gplay_Reviews_all.apply(lambda x: TextBlob(x).words)

In [ ]:
Gplay_Reviews_all

0       assalamu alaikum software really helpful top d...
1       app good transactions bug entry something stoc...
2       amazing app use app monthsvery helpful easy un...
3       bad app bad service far none complaints receiv...
4       well present services bad topup taking much ti...
                              ...                        
3684                                                 good
3685                                                 nice
3686                                                 good
3687                                                 nice
3688                                           asmotullah
Name: content, Length: 3689, dtype: object

In [ ]:
Gplay_Reviews_tokendf = Gplay_Reviews_token.to_frame()

In [ ]:
len(Gplay_Reviews_token)

3689

In [ ]:
Gplay_Reviews_tokendf['content']

0       [assalamu, alaikum, software, really, helpful,...
1       [app, good, transactions, bug, entry, somethin...
2       [amazing, app, use, app, monthsvery, helpful, ...
3       [bad, app, bad, service, far, none, complaints...
4       [well, present, services, bad, topup, taking, ...
                              ...                        
3684                                               [good]
3685                                               [nice]
3686                                               [good]
3687                                               [nice]
3688                                         [asmotullah]
Name: content, Length: 3689, dtype: object

In [ ]:
tf = Gplay_Reviews_all

In [ ]:
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    # traverse in the string  
    for ele in s: 
        str1 += str(ele)  
    
    # return string  
    return str1 

In [ ]:
def word_count(str):
    counts = dict()
    words = str.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

In [ ]:
len(tf)

3689

In [ ]:
Gplay_count = word_count(listToString(tf))

In [ ]:
Gplay_count

{'assalamu': 1,
 'alaikum': 1,
 'software': 8,
 'really': 5,
 'helpful': 13,
 'top': 18,
 'different': 2,
 'many': 7,
 'good': 56,
 'programs': 1,
 'surprised': 1,
 'see': 2,
 'biggest': 2,
 'thing': 1,
 'anyone': 1,
 'wants': 3,
 'business': 49,
 'manager': 10,
 'call': 18,
 'customer': 10,
 'care': 3,
 'want': 4,
 'know': 7,
 'setup': 1,
 'use': 42,
 'without': 6,
 'problem': 12,
 'benefit': 1,
 'lotapp': 1,
 'transactions': 1,
 'bug': 4,
 'entry': 3,
 'something': 6,
 'stock': 3,
 'payment': 13,
 'made': 2,
 'unchanged': 1,
 'even': 6,
 'delete': 10,
 'edit': 6,
 'app': 255,
 'inventory': 1,
 'pos': 2,
 'big': 5,
 'blunder': 1,
 'hope': 4,
 'developer': 1,
 'updateamazing': 1,
 'monthsvery': 1,
 'easy': 11,
 'understand': 4,
 'everyone': 2,
 'gives': 3,
 'much': 15,
 'profit': 3,
 'im': 10,
 'completely': 2,
 'satisfiedbad': 1,
 'bad': 24,
 'service': 39,
 'far': 1,
 'none': 1,
 'complaints': 1,
 'received': 2,
 'resolved': 1,
 'lost': 2,
 'also': 10,
 'pay': 7,
 'appwell': 1,
 'pre

In [ ]:
Gplay_Final = pd.DataFrame.from_dict([Gplay_count],)

In [ ]:
Gplay_final = pd.DataFrame(Gplay_count.items(), columns=['Comments', 'Count'])

In [ ]:
Gplay_final

,Comments,Count
0,assalamu,1
1,alaikum,1
2,software,8
3,really,5
4,helpful,13
...,...,...
5315,hapnice,1
5316,postfakebadfoulbadcheatingfraudrubellanice,1
5317,appnicebeautifulgoodexcellentgoodgoodgoodniceg...,1
5318,appnicenicenicegoodnicenicenicenicenicenicenic...,1


In [ ]:
Gplay_final.to_csv('Google_Play_sManager.csv')